In [1]:
!module add hdp
!cypress-kinit

In [2]:
!hdfs dfs -ls -h /repository/movielens  #the movieLen dataset was update in March 2017.

Found 7 items
-rw-r--r--   2 lngo hdfs-user      9.3 K 2017-03-15 09:49 /repository/movielens/README.txt
-rw-r--r--   2 lngo hdfs-user    317.9 M 2017-03-15 09:49 /repository/movielens/genome-scores.csv
-rw-r--r--   2 lngo hdfs-user     17.7 K 2017-03-15 09:49 /repository/movielens/genome-tags.csv
-rw-r--r--   2 lngo hdfs-user    839.2 K 2017-03-15 09:49 /repository/movielens/links.csv
-rw-r--r--   2 lngo hdfs-user      1.9 M 2017-03-15 09:49 /repository/movielens/movies.csv
-rw-r--r--   2 lngo hdfs-user    632.7 M 2017-03-15 09:49 /repository/movielens/ratings.csv
-rw-r--r--   2 lngo hdfs-user     22.9 M 2017-03-15 09:49 /repository/movielens/tags.csv


In [6]:
%%writefile solution/userMapper.py
#!/usr/bin/env python

import sys
import csv
import json

movieFile = "./movielens/movies.csv"
movieList = {}

with open(movieFile, mode = 'r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        movieList[row[0]] = {}
        movieList[row[0]]["title"] = row[1]
        movieList[row[0]]["genre"] = row[2].strip().split("|")

userDict = {}
        
for oneMovie in sys.stdin:
    oneMovie = oneMovie.strip()
    ratingInfo = oneMovie.split(",")
    try:
        user = int(ratingInfo[0])
        movieTitle = movieList[ratingInfo[1]]["title"]
        movieGenre = movieList[ratingInfo[1]]["genre"]
        rating = float(ratingInfo[2])
        
        if user not in userDict:
            userDict[user] = []
        userDict[user].append(movieGenre)  
    except ValueError:
        continue
        
for user in userDict:
    print("%s\t%s" % (user, json.dumps(userDict[user])))

Overwriting solution/userMapper.py


In [4]:
%%writefile solution/userReducer.py 
#!/usr/bin/env python

import sys
import csv
import json

mostRatingUser = None
mostRatingCount = 0
mostRatingInfo = None

for line in sys.stdin:
    line = line.strip()
    user, genreString = line.split("\t", 1)
    genreInfo = json.loads(genreString)

    if not mostRatingUser or len(genreInfo) > mostRatingCount:
        mostRatingUser = user
        mostRatingCount = len(genreInfo)
        mostRatingInfo = genreInfo
        
# print(mostRatingCount)

genreDict = {}
for genreList in mostRatingInfo:
    for genre in genreList:    
        if genre not in genreDict:
            genreDict[genre] = 0
        genreDict[genre] += 1 

mostRatedCount = 0
mostRatedGenre = None

for genre, count in genreDict.items():
    if count > mostRatedCount:
        mostRatedCount = count
        mostRatedGenre = genre
    
print("%s -- Total Rating Counts: %d -- Most Rated Genre: %s - %d" % (mostRatingUser, mostRatingCount, mostRatedGenre, mostRatedCount))

Writing solution/userReducer.py


In [7]:
!hdfs dfs -cat /repository/movielens/ratings.csv 2>/dev/null \
    | python ./solution/userMapper.py \
    | sort \
    | python ./solution/userReducer.py

186590 -- Total Rating Counts: 13250 -- Most Rated Genre: Drama - 8026
